# Filtering

## Hidden Markov models


Consider the evolution of the state $\{x_k\}$ and the partial observation $\{y_k\}$ at time $k$
$$ 
\begin{align*}
x_{k+1} = f(x_k) + \omega_{k+1}\\
y_{k+1} = h(x_k) + \nu_{k+1}
\end{align*}
$$
where evolution error $\omega_{k+1}$ and observation error $\nu_{k+1}$ are drawn from Gaussian distributons of known covarainces $\Sigma_{\omega}$ and $\Sigma_{\nu}$.



## Filtering algorithm

Different filtering algorithms are designed to estimate the distribution of the state $\{x_k\}$
$$p(x_k|Y_{k}) \qquad \textrm{where} \qquad Y_k = \{y_1, y_2,\cdots, y_k\}$$

The prediction step involves the Chapman-Kolmogorov equation 
$$ 
p(x_{k+1}|Y_{k}) = \int p(x_{k+1}|x_k) p(x_{k}|Y_{k}) dx_{k} = \int \frac{1}{\sqrt{(2\pi)^{N_{x}}|\Sigma_{\omega}|}} e^{-\frac{1}{2}(x_{k+1} - f(x_{k}))^T\Sigma_{\omega}^{-1}(x_{k+1} - f(x_{k}))} p(x_{k}|Y_{k}) dx_{k} \\
$$
The analysis step involves Bayes' rule
$$ 
p(x_{k+1}|Y_{k+1}) = p(x_{k+1}|y_{k+1}, Y_{k}) = \frac{p(y_{k+1}|x_{k+1})p(x_{k+1}|Y_{k})}{p(y_{k+1}|Y_k)} \propto e^{-\frac{1}{2}(y_{k+1} - h(x_{k+1}))^T\Sigma_{\nu}^{-1}(y_{k+1} - h(x_{k+1}))}p(x_{k+1}|Y_{k})
$$

Combining prediction and analysis steps leads to the recursive relation $p(x_{k}|Y_{k}) \rightarrow p(x_{k+1}|Y_{k+1})$

# Particle Filtering 

## Sequential Importance Sampling
Consider the full posterior probability density $p(x_{0:k}| Y_k)$ given the measurements $Y_k$, we have the following recursion
$$p(x_{0:k}|Y_k) \propto p(y_k|x_k) p(x_k|x_{k-1})p(x_{0:{k-1}}|Y_{k-1}) $$

To sample $x_{0:k}$, we could first construct an importance distribution $x_{0:k}^{j} \sim \pi(x_{0:k}|Y_k)$ and compute the corresponding imporance weights as 
$$\omega_k^{j} \propto \frac{p(y_k|x_k^j) p(x_k^j|x_{k-1}^j)p(x_{0:{k-1}}^j|Y_{k-1})}{\pi(x_{0:k}^j|Y_k)} $$

To sequentially sample $x_{0:k}$, let's form the importance distribution recursively as follows
$$\pi(x_{0:k}|Y_k) = \pi(x_k|x_{0:k-1},Y_k) \pi(x_{0:k-1}|Y_{k-1})$$

In the prediction step, we draw $J$ new samples $x_k^j$ from importance distributions
$$x_k^j \sim \pi(x_k|x_{0:k-1},Y_k)$$
In the analysis step, we update the weights
$$\omega_k^{j} \propto \frac{p(y_k|x_k^j) p(x_k^j|x_{k-1}^j)p(x_{0:{k-1}}^j|Y_{k-1})}{\pi(x_{0:k}^j|Y_k)} = \frac{p(y_k|x_k^j) p(x_k^j|x_{k-1}^j)}{\pi(x_{k}^j|x_{0:k-1}^j, Y_k)}\omega_{k-1}^j$$

Resampling is required to to avoid converging to single non-zero weight $\omega^i = 1$, which can be monitored by the effective number of samples 
$$N_{eff} = \frac{1}{\sum_{j=1}^J(\omega_k^{j})^2}$$

### Bootstrap Filter
In bootstrap filter, we use dynamic model as the importance distribution
$$\pi(x_{k}^j|x_{0:k-1}^j, Y_k) \propto p(x_{k}^j|x_{k-1}^j) \omega_{k-1}^j$$

We will draw points $\{x_{k}^j\}$ from the dynamic model:
$$x_{k}^j \sim p(x_k | x_{k-1}^{j})$$
And calculate new weights 
$$\omega_k^{j} \propto p(y_k | x_{k}^{j})$$


# Kalman Filtering

Kalman filtering can be viewed as the following recursive relationship
$$
\begin{align*}
&p(x_k|Y_{k}) = \mathcal{N}(m_k, C_k)\\
&p(x_{k+1}|Y_{k}) = \mathcal{N}(\hat{m}_{k+1}, \hat{C}_{k+1})\\
&p(x_{k+1}|Y_{k+1}) = \mathcal{N}(m_{k+1}, C_{k+1})
\end{align*}
$$

The prediction step estimates
$$
\begin{align*}
\hat{m}_{k+1} = \mathbb{E}[x_{k+1} | Y_{k+1}] &= \int\int x_{k+1} p(x_{k+1}|x_k) p(x_k|Y_k) d x_k  dx_{k+1} \\
                                             &= \int f(x_{k}) p(x_k|Y_k) d x_k \\
                                             & = \mathbb{E}[f(x_{k}) | Y_{k}] \\
\hat{C}_{k+1} = \textrm{Cov}[x_{k+1} | Y_{k+1}] &= \int\int (x_{k+1} - \hat{m}_{k+1})(x_{k+1} - \hat{m}_{k+1})^T p(x_{k+1}|x_k) p(x_k|Y_k) d x_k  dx_{k+1} \\
                                             &= \int \Bigl[f(x_{k}) f(x_{k})^T + \Sigma_{\nu} \Bigr] p(x_k|Y_k) d x_k  - \hat{m}_{k+1} \hat{m}_{k+1}^T\\
                                             & = \mathrm{Cov}[f(x_{k}) | Y_{k}] + \Sigma_{\nu}\\
\end{align*}
$$

The analysis step first apprximates the joint distribution of $p(x_{k+1}, y_{k+1} | Y_{k})$ as a Gaussian distribution
$$\begin{equation}
     \mathcal{N}\Bigl(
    \begin{bmatrix}
    \hat{m}_{k+1}\\
    \hat{y}_{k+1}
    \end{bmatrix}, 
    \begin{bmatrix}
   \hat{C}_{k+1} & \hat{C}_{k+1}^{x y}\\
    {\hat{C}_{k+1}^{x y}}{}^{T} & \hat{C}_{k+1}^{yy}
    \end{bmatrix}
    \Bigr).
\end{equation}$$

Then, with $\mathbb{E}$ denoting expectation with respect to 
$x_{k+1}|Y_k \sim \mathcal{N}( \hat{m}_{k+1},\hat{C}_{k+1})$, 

$$\begin{align*}
    \hat{y}_{k+1} =     & \mathbb{E}[h(x_{k+1})|Y_k], \\
    \hat{C}_{k+1}^{x y} =     &  \mathrm{Cov}[x_{k+1}, h(x_{k+1})|Y_k],\\
    \hat{C}_{k+1}^{y y} = &  \mathrm{Cov}[h(x_{k+1})|Y_k] + \Sigma_{\nu}.
\end{align*}$$

Conditioning the Gaussian to find $x_{k+1}|\{Y_k,y_{k+1}\}=x_{k+1}|Y_{k+1}$ gives the following
expressions for the mean $m_{k+1}$ and covariance $C_{k+1}$ :

$$
\begin{equation}
    \begin{split}
        m_{k+1} &= \hat{m}_{k+1} + \hat{C}_{k+1}^{x y} (\hat{C}_{k+1}^{y y})^{-1} (y_{k+1} - \hat{y}_{k+1}),\\
        C_{k+1} &= \hat{C}_{k+1} - \hat{C}_{k+1}^{x y}(\hat{C}_{k+1}^{y y})^{-1} {\hat{C}_{k+1}^{x y}}{}^{T}.
    \end{split}
\end{equation}
$$

Different Kalman filters are designed for Gaussian integrations 
$$\mathbb{E}[f(x_{k}) | Y_{k}], \, \mathrm{Cov}[f(x_{k}) | Y_{k}],\, \mathbb{E}[h(x_{k+1})|Y_k],\,\mathrm{Cov}[x_{k+1}, h(x_{k+1})|Y_k],\,\textrm{and}\, \mathrm{Cov}[h(x_{k+1})|Y_k]$$


# Reference
1. [A Tutorial on Particle Filters for Online Nonlinear/Non-Gaussian Bayesian Tracking](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=978374)
2. [A Tutorial on Particle Filtering and Smoothing: Fifteen years later](https://warwick.ac.uk/fac/sci/statistics/staff/academic-research/johansen/publications/dj11.pdf)
3. [Nonlinear Bayesian Estimation: From Kalman Filtering to a Broader Horizon](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8283968)